# Adult Income Prediction Model

This notebook builds a machine learning pipeline to predict whether an individual's income exceeds $50K/year based on census data. We'll use XGBoost classifier with proper preprocessing for both numerical and categorical features.

## 1. Import Libraries

Loading essential libraries for data manipulation, visualization, and machine learning.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import pickle

#### Dataset Link: https://www.kaggle.com/datasets/uciml/adult-census-income/data

In [3]:
df = pd.read_csv('adult.csv',na_values=["?"])
df.drop(columns="fnlwgt",inplace=True)
df.head()

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


## 2. Data Cleaning

Handling missing values and removing duplicates to ensure data quality.

#### Using Ydata_Profiling for EDA

In [4]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df)
# profile.to_file("report.html")

### Remove Duplicate Rows

In [5]:
df.drop_duplicates(inplace=True)

### Check for Missing Values

In [6]:
df.isnull().sum()

age                  0
workclass         1632
education            0
education.num        0
marital.status       0
occupation        1639
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     580
income               0
dtype: int64

### Handle Missing Values

Filling missing values with appropriate strategies:
- **workclass** and **occupation**: Fill with 'Other' category
- **native.country**: Fill with mode (most common value)

In [7]:
df["workclass"].fillna("Other",inplace=True)

C:\Users\Subhan Ahmed Chandio\AppData\Local\Temp\ipykernel_13328\2983916311.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["workclass"].fillna("Other",inplace=True)


In [8]:
df["workclass"].value_counts()

workclass
Private             19621
Self-emp-not-inc     2473
Local-gov            2040
Other                1632
State-gov            1272
Self-emp-inc         1091
Federal-gov           946
Without-pay            14
Never-worked            7
Name: count, dtype: int64

In [9]:
df["occupation"].fillna("Other",inplace=True)

C:\Users\Subhan Ahmed Chandio\AppData\Local\Temp\ipykernel_13328\114737413.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["occupation"].fillna("Other",inplace=True)


In [10]:
df["native.country"].mode()

0    United-States
Name: native.country, dtype: object

In [11]:
df["native.country"] = df["native.country"].fillna(df["native.country"].mode()[0])

### Verify No Missing Values Remain

In [12]:
df.isnull().sum()

age               0
workclass         0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [13]:
df.sample(5)

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
32435,31,Private,HS-grad,9,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,35,United-States,<=50K
17523,73,Local-gov,Some-college,10,Never-married,Prof-specialty,Other-relative,White,Female,0,0,11,United-States,<=50K
12275,39,Private,10th,6,Divorced,Other-service,Not-in-family,White,Female,0,0,60,United-States,<=50K
22656,64,Other,Some-college,10,Widowed,Other,Unmarried,White,Female,0,0,8,United-States,<=50K
14721,18,Other,Some-college,10,Never-married,Other,Own-child,White,Female,0,0,4,United-States,<=50K


## 3. Machine Learning Pipeline

Building an end-to-end ML pipeline with preprocessing and model training.

### Prepare Features and Target Variable

Separating features (X) and target (y). Converting income to binary: 1 for >50K, 0 for <=50K.

In [14]:
X = df.drop("income",axis=1)
y = df["income"].apply(lambda x: 1 if ">50K" in x else 0)

### Split Data into Training and Testing Sets

Using 80-20 split for training and testing.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape

((23276, 13), (5820, 13))

In [16]:
df.columns

Index(['age', 'workclass', 'education', 'education.num', 'marital.status',
       'occupation', 'relationship', 'race', 'sex', 'capital.gain',
       'capital.loss', 'hours.per.week', 'native.country', 'income'],
      dtype='object')

### Define Preprocessing Pipeline

Creating a ColumnTransformer to handle different feature types:
- **Numerical features**: Standardized using StandardScaler
- **Ordinal features**: education.num (already encoded 1-16, passed through)
- **Categorical features**: One-hot encoded with first category dropped to avoid multicollinearity

In [ ]:
# If using education-num instead of education text column
numerical_columns = ["age", "capital.gain", "capital.loss", "hours.per.week"]
ordinal_columns = ["education.num"]  # Already encoded 1-16
nominal_columns = ["workclass", "marital.status", "occupation", 
                   "relationship", "race", "sex", "native.country"]

transformer = ColumnTransformer([
    ("num_scaler", StandardScaler(), numerical_columns),
    ("ordinal_pass", 'passthrough', ordinal_columns),  # No need to encode
    ("nominal_encoder", OneHotEncoder(drop="first", sparse_output=False,handle_unknown='ignore'), nominal_columns)
], remainder='drop')

### Pipeline

In [ ]:
pipe = Pipeline([
    ("pre-processing",transformer),
    ("model",XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
))
])

In [19]:
pipe.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('pre-processing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num_scaler', ...), ('ordinal_pass', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the di

#### Model Prediction & Evaluation

In [20]:
y_pred = pipe.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nPipeline Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Cross-validation
cv_scores = cross_val_score(pipe, X_train, y_train, cv=10, scoring='accuracy')
print(f"Cross-Validation Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

c:\Users\Subhan Ahmed Chandio\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:261: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)



Pipeline Accuracy: 0.8641 (86.41%)
Cross-Validation Score: 0.8693 (+/- 0.0054)


## Cross-Validation Score: 0.8693

#### model saving 

In [21]:
pickle.dump(pipe, open('pipe.pkl', 'wb'))